<a href="https://colab.research.google.com/github/devt287/applied_stat/blob/main/HW4_P3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install necessary packages; this may take some time
# "JM" contains the "aids" dataset we will be using for this coding problem
# "survminer" contains many useful functions for the survival analysis

install.packages("JM")
install.packages("survminer")
library("JM")
library("survminer")

In [ ]:
# we will be using the "aids.id" dataset; look at the first few rows for an overview
# the explanations of the variables can be found at https://rdrr.io/cran/JM/man/aids.html
head(aids.id)

# YOUR CODE (~1-3 lines): from the aids.id data,
# plot the Kaplan-Meier curves for males and females (ignoring all other features), separately
# you should include both curves in the same plot, as well as confidence intervals
# note 1: it might be helpful to first construct a survival object using the "Surv" function
# note 2: it might be helpful to look up the "ggsurvplot" and "survfit" functions
# note 3: there might be more than one ways to implement it (using different functions/packages),
# and they are all fine as long as you are plotting the correct thing


In [ ]:
# YOUR CODE (~1 line):
# fit a Cox model with "gender" being the only feature
# look at the fitted coefficient, as well as its significance
cox_model =
summary(cox_model)

# YOUR CODE (~3 lines):
# plot the fitted survival curves for males and females based on the Cox model
# you should include both curves in the same plot, as well as confidence intervals
# look at the differences from the above Kaplan-Meier curves
# note: the "ggsurvplot_combine" function might be useful to combine two plots


In [ ]:
# YOUR CODE (~1-2 lines):
# plot the Kaplan-Meier curves for all 4 possible combinations of (drug, prevOI), ignoring other features
# you should include all curves in the same plot (no confidence intervals needed)
# based on your plot, understand the effects of (drug, prevOI)


In [ ]:
# YOUR CODE (~1 line):
# fit a Cox model with two features (drug, prevOI)
# look at the fitted coefficients and their significance levels

cox_model2 =
summary(cox_model2)

# YOUR CODE (~5 lines):
# plot the fitted curves for all 4 possible combinations of (drug, prevOI)
# you should include all curves in the same plot (no confidence intervals needed)
# look at the differences from the above Kaplan-Meier curves


In [ ]:
# YOUR CODE (~1 line):
# fit another Cox model with features (drug, prevOI), as well as the interaction drug * prevOI
# look at the fitted coefficients and their significance levels

cox_model3 =
summary(cox_model3)

# YOUR CODE (~5 lines):
# plot the fitted curves for all 4 possible combinations of (drug, prevOI)
# you should include all curves in the same plot (no confidence intervals needed)
# look at the differences from the above Cox model without interaction
